In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np
from PolicyGrad import Policy, fic_replay_bp_ddqn

In [ ]:
from simulation_lib import *

# Set global digit printing options
np.set_printoptions(precision=4)

# Define Training and Model Parameters
num_players = 5           # Total number of agents

# Default simulation parameters

kappa = 0.5
sim_dict = {
        'perm_price_impact': torch.tensor(0.05).cuda(),
        'transaction_cost': torch.tensor(.1).cuda(),
        'liquidation_cost': torch.tensor(.1).cuda(),
        'running_penalty': torch.tensor(0.0).cuda(),
        'trans_impact_scale':torch.tensor(0.02).cuda(),
        'trans_impact_decay':torch.tensor(0.5).cuda(),
        'T': torch.tensor(5).cuda(),
        'dt': torch.tensor(0.5).cuda(),
        'N_agents': num_players,
        'drift_function': (lambda x, y: kappa*(10-y)),
        'volatility': torch.tensor(0.1).cuda(),
        #'initial_price_var': .2,
        'init_inv_var': torch.tensor(50).cuda()}

# compute invariant distribution for initial price variance
inv_std = sim_dict['volatility']* torch.sqrt((1 - torch.exp(-2*kappa*sim_dict['T']))/ (2*kappa))
sim_dict['initial_price_var'] = torch.tensor(inv_std).cuda()

norm_mean = torch.tensor([2.25, 10, 0, 0 ]).cuda()
norm_std = torch.tensor([1.4361406616345072, 0.74204157112471332, 4.512414940762905, 0.33068192958806433 ]).cuda()

sim_obj = MarketSimulator(sim_dict, store_hist = False, impact='sqrt')

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
pol, val, pol_t_loss_arr, val_t_loss_arr = fic_replay_bp_ddqn(sim_obj, norm_mean, norm_std,  its=20, tau=0.5, it_lim=5000,start_itn=10000, lr=3e-4, weight_decay=1e-3, other_pol_init=None, starting_i=0, wdir='/pt_files/Fic_play_ddqn/', q0=True, minibatch=10)